## Compute tick count using dedicated service(F#)

### Overview
Tick count can be used as an indicator to illustrate market activity by computing the technical transactions/transmissions count on a tick by tick basis.

It represents a valuable technical indicator to monitor tick by tick data reception. Building a history of tick count can be useful in the following use-cases:
* guarantee tick by tick data good reception and completion
* compare trading periods and highlight high-level of activity periods
* combined with other indicators, it helps to optimal future rolling 
* etc.

Correlated with a set of other indicators such as traded volume and count of trades, over the same look back period, it repserents a good measurement of the market quality.

This samples enables to retrieve **on-demand** tick count data points by calling a dedicated service.

### Inputs/outputs
Tick count sample requires instrument's identifier, date time intervals and time granularity as per inputs. It returns the tick count sampled using the input time granularity.

This sample also shows how to plot a simple graph for basis financial analysis using an open source library.


### Services used
This sample uses *gRPC requests* in order to retrieve ticks from the dedicated hosted service. The queried endpoint in this script are:
* *TopologiesService*: to directly retrieve ticks objects from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Topology.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run tick count sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre TaskBuilder.fs, 2.1.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Topology.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *TopologiesService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Topology request parameters

#### 3.1 Instrument selection

In [4]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Topology parmeters

In [5]:
let granularity = TopologyGranularity.Daily
let level = Level.TradesAndBook

#### 3.2 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [6]:
// Generate the topologies request
let request = 
    new TopologiesRequest( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Granularity = granularity ,
        Level = level
    )

// Instantiate the topologies service
let service = new TopologiesService.TopologiesServiceClient(channel)

### Step4: Visualize data

#### 4.1 Retrieve tick count data

In [7]:
// Process the topologies request
let reply = service.Topologies(request, headers)

// Generate request response and total tick count
let topologies = reply.Entries
let totalTicks = reply.Entries |> Seq.sumBy (fun t -> t.TicksCount)

// Retrieve daily tick count
let topologiesSelection = topologies |> Seq.map (fun t -> (new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount))

#### 4.2 Plot tick count
Plot daily tick count over the look back period from the request result and plot them with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.

In [8]:
topologiesSelection
|> Chart.Column
|> Chart.WithTitle $"{granularity} ticks count for {ticker} - Total ticks count: {totalTicks:N0}"
|> Chart.WithYTitle "#Ticks"
|> Chart.WithXTitle "Time"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display

<!DOCTYPE html>